In [1]:
###Load Packages
library(viridis)
library(ggplot2)
library(gridExtra)
library(grid)
library(RColorBrewer)
library(dplyr)







Attaching package: ‘dplyr’

The following object is masked from ‘package:gridExtra’:

    combine

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



In [2]:
library(grDevices)


In [3]:
library("IRdisplay")

In [4]:
library("circlize")

In [5]:
#library("ggfortify")

In [6]:
library(OmicCircos)

Loading required package: GenomicRanges
Loading required package: BiocGenerics
Loading required package: parallel

Attaching package: ‘BiocGenerics’

The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB

The following objects are masked from ‘package:dplyr’:

    combine, intersect, setdiff, union

The following object is masked from ‘package:gridExtra’:

    combine

The following objects are masked from ‘package:stats’:

    IQR, mad, xtabs

The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, cbind, colnames, do.call,
    duplicated, eval, evalq, Filter, Find, get, grep, grepl, intersect,
    is.unsorted, lapply, lengths, Map, mapply, match, mget, order,
    paste, pmax, pmax.int, pmin, pmin.int, Position, rank, rbind,
    Reduce, rownames, sapply, setdiff, sort

In [7]:
#library("genoPlotR")

In [8]:
#library("dplyr")

In [9]:
#library("ggbio")

## Set working dir path

In [10]:
path <-"."


### define a function for multiple plots with a single legend

In [11]:
##A function to enable multiple plots that share a single legend
##See https://github.com/hadley/ggplot2/wiki/Share-a-legend-between-two-ggplot2-graphs

    grid_arrange_shared_legend <- function(..., position = c("bottom", "right")) {

  plots <- list(...)
  position <- match.arg(position)
  g <- ggplotGrob(plots[[1]] + theme(legend.position=position))$grobs
  legend <- g[[which(sapply(g, function(x) x$name) == "guide-box")]]
  lheight <- sum(legend$height)
  lwidth <- sum(legend$width)
  gl <- lapply(plots, function(x) x + theme(legend.position="none"))

  combined <- switch(position,
                     "bottom" = arrangeGrob(do.call(arrangeGrob, gl),
                                            legend,
                                            ncol = 1,
                                            heights = unit.c(unit(1, "npc") - lheight, lheight)),
                     "right" = arrangeGrob(do.call(arrangeGrob, gl),
                                           legend,
                                           ncol = 2,
                                           widths = unit.c(unit(1, "npc") - lwidth, lwidth)))
  grid.newpage()
  grid.draw(combined)
}

### specific color pallete for regions

In [12]:

color <- c("#ffcc00", "#ff0000", "#00ffbf", "#0000ff", "#000066", "#660066", "#006600", "#66ff66")
Region <- c("IO", "RS", "MS", "NPO", "SPO", "SO", "SAO", "NAO")
color_region <- data.frame(color, Region)


### set contigs groups, lengths

In [96]:
genome_name <- c("contig_155688", "contig_154566", "contig_156409",  "contig__154592", "contig__154680",
             "contig__154676", "contig__155773", "contig__172037", "contig_154940", "contig__155057",
            "SAMEA2621151_40", "SAMEA2621061_122", "SAMEA2621085_65", "SAMEA2619923_17", "SAMEA2621151_104",
            "SAMEA2620879_23", "SAMEA2620078_38")

#contig__154680!! contig__154676!! SAMEA2621061_122!! SAMEA2621151_40!! SAMEA2621085_65!!! contig__154566!! SAMEA2619923_17!! contig__154592!!
#contig__155688!! contig__155057!! NA contig__156409!! contig__155773!! SAMEA2620879_23!! SAMEA2621151_104!!! contig__17203!!7

length  <- c("104484","105801","118409","96096","96282","53712","84959","61133","60318","63765","92290",
            "106637","102641","102641","92290","93879","94102")

magrovirus_group <- c("A","A","A","B1","B1","B1","B2","C","C","C","B2","B2","B2","B2","A","A","A")

contig_lengths <- data.frame(genome_name, length, magrovirus_group)





In [14]:
#contig_lengths

# Genome Maps 

In [18]:
path_maps <- paste(path,'/', sep = '')
group_A_map_path<- paste(path,'/group_A_annotations_for_circle.txt', sep = '')
group_A_map_df <- read.csv(group_A_map_path, sep='\t', stringsAsFactors=FALSE)

#get just the eilat contigsw
group_A_map_df_eilat <- group_A_map_df[grep("contig", group_A_map_df$chr),  ]

#load orf annotation 
orf_annotations_groups_path <- paste(path, '/orf_annotations_groups.txt', sep='')
orf_annotations_groups <- read.csv(orf_annotations_groups_path, sep='\t')


### load groupA eilat contigs blastP results

In [62]:
all_vs_all_blastp_genes_start_end_path <- paste(path,'/all_vs_all_blastp_genes_start_end.tsv', sep='')
all_v_all_blastp_results_groupA_blast_start_end_path <- paste(path,'/all_v_all_blastp_results_groupA_blast_start_end.tsv', sep='')
all_v_all_blastp_results_groupA_blast_start_end <- read.csv(all_v_all_blastp_results_groupA_blast_start_end_path, sep='\t', stringsAsFactors=FALSE)



### load groupA all contigs blastP results


In [61]:
all_vs_all_blastp_genes_start_end_eilat_path <- paste(path,'/all_v_all_plus_annotation_start_end.tsv', sep='')
all_v_all_blastp_results_groupA_blast_start_end_path <- paste(path,'/all_v_all_blastp_results_groupA_blast.tsv', sep='')
all_v_all_blastp_results_groupA_blast_start_end <- read.csv(all_v_all_blastp_results_groupA_blast_start_end_path, sep='\t', stringsAsFactors=FALSE)



# genome maps with Circlize


#### first create a color pallete for the genes

In [44]:
#get unique values of orf_annotations
annotations <- unique(orf_annotations_groups$group)

In [45]:
total <- merge(group_A_map_df,orf_annotations_groups,by="orf_annotation", )

In [46]:
annotations

[1] Hypothetical protein                                
 [2] Structural protein                                  
 [3] Chaperonin                                          
 [4] Transcription, Replication, recombination and repair
 [5] CRISPR/Cas system-associated protein                
 [6] DNA-binding protein                                 
 [7] General function prediction                         
 [8] DUF                                                 
 [9] DNA packaging                                       
[10] Amino acid degradation                              
[11] host recognition                                    
[12] DNA ejection                                        
[13]                                                     
13 Levels:  Amino acid degradation ... Transcription, Replication, recombination and repair

In [47]:
annotations <- as.character(annotations)

In [48]:
#from some reasong there's a 13th null element. remove it.
annotations <- annotations[-13]
annotations

[1] "Hypothetical protein"                                
 [2] "Structural protein"                                  
 [3] "Chaperonin"                                          
 [4] "Transcription, Replication, recombination and repair"
 [5] "CRISPR/Cas system-associated protein"                
 [6] "DNA-binding protein"                                 
 [7] "General function prediction"                         
 [8] "DUF"                                                 
 [9] "DNA packaging"                                       
[10] "Amino acid degradation"                              
[11] "host recognition"                                    
[12] "DNA ejection"

In [49]:
#create a custom color palette
annotation_color <- brewer.pal(length(annotations), "Paired")


In [50]:
#add group names to colors
names(annotation_color) <- annotations
contig_col <- brewer.pal(8, "Set1")

In [51]:
annotation_color

Hypothetical protein 
                                           "#A6CEE3" 
                                  Structural protein 
                                           "#1F78B4" 
                                          Chaperonin 
                                           "#B2DF8A" 
Transcription, Replication, recombination and repair 
                                           "#33A02C" 
                CRISPR/Cas system-associated protein 
                                           "#FB9A99" 
                                 DNA-binding protein 
                                           "#E31A1C" 
                         General function prediction 
                                           "#FDBF6F" 
                                                 DUF 
                                           "#FF7F00" 
                                       DNA packaging 
                                           "#CAB2D6" 
                              Amino acid degradation 
                                           "#6A3D9A" 
                                    host recognition 
                                           "#FFFF99" 
                                        DNA ejection 
                                           "#B15928"

In [65]:
#typeof(
annotation_col <- data.frame(annotations,annotation_color, stringsAsFactors = FALSE)
colnames(annotation_col) <- c("group","col")
rownames(annotation_col) <- annotation_col[,1]
#annotation_col[,1] <- NULL


In [86]:
#group_A_map_df

In [87]:
#create a sector list ()
sector <- unique(group_A_map_df$name)
sector <- as.character(sector)


In [97]:
#defien the xlim - the size of each secotr
sector_xlim = NULL
for (t in unique(group_A_map_df[[1]])) {
    print(t)
    sector_xlim = rbind(sector_xlim, c(0, as.double(contig_lengths[contig_lengths[1]==t][2])))
}

[1] "contig_156409"
[1] "SAMEA2621061_122"
[1] "contig_154566"
[1] "contig_155688"
[1] "SAMEA2619923_17"
[1] "SAMEA2621085_65"
[1] "SAMEA2621151_40"


In [98]:
sector_xlim

0,118409
0,106637
0,105801
0,104484
0,102641
0,102641
0,92290


In [99]:
#make colnames readable for circlize

colnames(group_A_map_df) <- c("name","gene_name","start","end","length","strand","orf_annotation","conserved_proteins")

group_A_map_df_circ <- total[,c(2,4,5,6,3,7,1,8,9)]
#head(group_A_map_df_circ)

In [100]:

#"bending.outside"

face <- as.character(c("bending.inside", "bending.inside", "bending.outside", "bending.inside", "bending.inside",
                       "bending.outside", "bending.inside"))
adj <- c(-3,-3,3,3,-3,3, 3)
sector_face <- data.frame(sector, face,adj)


In [101]:
annotation_col


,group,col
Hypothetical protein,Hypothetical protein,#A6CEE3
Structural protein,Structural protein,#1F78B4
Chaperonin,Chaperonin,#B2DF8A
"Transcription, Replication, recombination and repair","Transcription, Replication, recombination and repair",#33A02C
CRISPR/Cas system-associated protein,CRISPR/Cas system-associated protein,#FB9A99
DNA-binding protein,DNA-binding protein,#E31A1C
General function prediction,General function prediction,#FDBF6F
DUF,DUF,#FF7F00
DNA packaging,DNA packaging,#CAB2D6
Amino acid degradation,Amino acid degradation,#6A3D9A


In [ ]:
par(family = 'sans')
all_v_all_blastp_results_groupA_blast_start_end$qseqid <- as.character(all_v_all_blastp_results_groupA_blast_start_end$qseqid)
group_A_map_df_circ$group <- as.character(group_A_map_df_circ$group)
#circos.par(cell.padding = c(0, 0, 0, 0), start.degree = 270, gap.degree = c(2, 2, 2))
circos.par(cell.padding=c(0,0,0,0), track.margin=c(0,0.15), start.degree = 90, gap.degree =4)
circos.initialize(factors = factor(sector, levels = sector), xlim = sector_xlim, )

##create outer tracks - genome color
circos.trackPlotRegion(ylim = c(0, 1), panel.fun = function(x, y) {
    sector.index = get.cell.meta.data("sector.index")
    xlim <- get.cell.meta.data("xlim")
    ylim <- get.cell.meta.data("ylim")
    #print (xlim)
    sub <- subset(group_A_map_df_circ,  name ==sector.index)
    for(i in seq_len(nrow(sub))) {
        circos.rect(sub[i,2], 0, sub[i,3], 1,
                sector.index = sub[i,1], col = annotation_col$col[annotation_col$group == sub[i,9]])
    
    }
    #get the index of the colums in the mauve backbone df
    
circos.par(track.margin=c(0,3)) 
    
    face = sector_face[sector_face[1]== sector.index][2]
    adj = as.numeric(sector_face[sector_face[1]== sector.index][3])
    circos.text(mean(xlim), mean(ylim+3), sector.index, cex = 0.7, facing = face, niceFacing = TRUE)#, adj = adj)
    circos.axis(h = "top", labels.cex = 0.4, major.tick.percentage = 0.3)
}, track.height = 0.08, bg.col = '#f2f2f2', track.margin = c(0.4, 0.2))

    

#}}

    
    
for(i in seq_len(nrow(all_v_all_blastp_results_groupA_blast_start_end))) {

    
        circos.par(track.margin=c(0,0))

        query_name = all_v_all_blastp_results_groupA_blast_start_end$query_contig[i]
        #print(all_v_all_blastp_results_groupA_blast_start_end$query_end[i])
        subject_name = all_v_all_blastp_results_groupA_blast_start_end$subject_contig[i]
 
            circos.link(query_name, c(all_v_all_blastp_results_groupA_blast_start_end$query_start[i], all_v_all_blastp_results_groupA_blast_start_end$query_end[i]),
        subject_name,
        c(all_v_all_blastp_results_groupA_blast_start_end$subject_start[i], all_v_all_blastp_results_groupA_blast_start_end$subject_end[i]),
        #col = annotation_col$col[annotation_col$group == group_A_map_df_circ$group[group_A_map_df_circ$name == all_v_all_blastp_results_groupA_blast_start_end$qseqid[i]]],
        border = NA)




#}
}
circos.clear()


In [104]:
all_v_all_blastp_results_groupA_blast_start_end

qseqid,sseqid,pident,length,mismatch,gapopen,qstart,qend,sstart,send,evalue,bitscore,query_contig,subject_contig,query_start,query_end,subject_start,subject_end
gene_103_contig_154566,gene_4_contig_156409,54.688,128,50,3,174,294,131,257,1.34e-27,112.0,contig_154566,contig_156409,96497,97951,4487,7084
gene_106_contig_154566,gene_63_contig_155688,53.846,104,48,0,10,113,1,104,4.76e-43,133.0,contig_154566,contig_155688,100796,101140,41810,42127
gene_11_contig_154566,gene_76_contig_155688,58.172,569,187,12,1,559,1,528,0.00e+00,633.0,contig_154566,contig_155688,19880,21562,67560,69149
gene_12_contig_154566,gene_78_contig_155688,65.079,63,22,0,36,98,21,83,3.01e-30,99.4,contig_154566,contig_155688,21562,21861,69776,70030
gene_13_contig_156409,gene_91_contig_155688,80.435,92,18,0,3,94,10,101,3.06e-50,151.0,contig_156409,contig_155688,17274,17573,84268,84672
gene_14_contig_156409,gene_93_contig_155688,95.652,46,2,0,1,46,7,52,2.56e-26,86.3,contig_156409,contig_155688,17785,17925,84872,85030
gene_15_contig_156409,gene_40_contig_154566,82.692,104,18,0,3,106,156,259,2.73e-56,171.0,contig_156409,contig_154566,17973,18293,48586,49365
gene_15_contig_156409,gene_66_contig_155688,80.952,105,15,1,2,106,13,112,7.02e-54,160.0,contig_156409,contig_155688,17973,18293,44124,44462
gene_17_contig_154566,gene_83_contig_155688,80.637,377,62,3,1,377,1,366,0.00e+00,613.0,contig_154566,contig_155688,23323,24456,71407,72507
gene_17_contig_154566,gene_88_contig_156409,69.531,384,107,6,1,377,1,381,0.00e+00,555.0,contig_154566,contig_156409,23323,24456,91376,92521
